### Initialization

In [ ]:
import pandas as pd
import eikon as ek
import datetime as dt

import os
from dotenv import load_dotenv

load_dotenv()
ek.set_app_key(os.getenv('EIKON_API_KEY'))
REPO_PATH = os.getenv('REPO_PATH')

### Daily Price data retrieval

In [ ]:
RIC = 'LCOc1'

START_DATE = dt.datetime(2023, 4, 11)
END_DATE = dt.datetime(2024, 4, 5)

dates = pd.date_range(START_DATE, END_DATE, freq='m')

price_df_list = []

from_date = START_DATE

run = True
while run:
    to_date = from_date + dt.timedelta(days=31)
    
    if to_date > END_DATE:
        to_date = END_DATE
        run = False

    price_data = ek.get_timeseries(
            RIC,
            start_date=str(from_date),
            end_date=str(to_date),
            interval='minute',
            calendar='calendardays',
            corax='adjusted'
        )

    price_df_list.append(price_data)
    from_date = price_data.index[-1]

price_df = pd.concat(price_df_list)
price_df = price_df[~price_df.index.duplicated()]

display(price_df)

### Save data

In [ ]:
file_name = f'{RIC}_High_Frequency.csv'

price_df.to_csv(
    f'{REPO_PATH}raw_futures_data/{file_name}', 
    index=True
)